In [314]:
import pandas as pd
import numpy  as np
excelf = pd.read_excel('/home/sara/Desktop/ITI_Python_for_Data_Management/Task 1/data/Energy Indicators.xls')
df = pd.DataFrame(excelf)

In [315]:
energy = df.iloc[17:244, 2:7]
energy=energy.rename(columns = {'Unnamed: 2':'Country', 'Unnamed: 3':'Energy Supply', 'Unnamed: 4':'Energy Supply per Capita','Unnamed: 5': '% Renewable'})
#print(energy.columns)
#energy

In [316]:
energy[['Energy Supply','Energy Supply per Capita','% Renewable']] =energy[ ['Energy Supply','Energy Supply per Capita','% Renewable'] ].replace('...',np.NaN) 

In [317]:
energy['Energy Supply'] = energy['Energy Supply'] * 1000000
#energy

In [334]:
energy['Country']=energy['Country'].replace({'Republic of Korea': 'South Korea',
'United States of America': 'United States',
'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
'China, Hong Kong Special Administrative Region': 'Hong Kong', 
'Iran, Islamic Rep.': 'Iran'})
energy['Country']=energy['Country'].str.replace(r"\(.*\)","")
energy['Country']=energy['Country'].str.replace('\d+','')
#energy

In [337]:
GDP = pd.read_csv('/home/sara/Desktop/ITI_Python_for_Data_Management/Task 1/data/world_bank.csv',skiprows=4)


In [338]:
GDP['Country Name'] = GDP['Country Name'].replace({'Korea, Rep.': 'South Korea', 
'Iran, Islamic Rep.': 'Iran',
'Hong Kong SAR, China': 'Hong Kong'})
#GDP

In [321]:
GDP = GDP[['Country Name','2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']]
#GDP.columns = [['Country','2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']]

In [322]:
ScimEn = pd.read_excel('/home/sara/Desktop/ITI_Python_for_Data_Management/Task 1/data/scimagojr-3.xlsx')
ScimEn = ScimEn[:15]
#ScimEn

In [323]:
#Step 3 
#Join the three datasets: GDP, Energy, and ScimEn into a new dataset 
#(using the intersection of country names). 
#Use only the last 10 years (2006-2015) of GDP data 
#and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).
mrg_df = pd.merge(ScimEn,energy,how='inner',left_on='Country',right_on='Country')
mrg_final = pd.merge(mrg_df,GDP,how='inner',left_on='Country',right_on='Country Name')
mrg_final = mrg_final.set_index('Country')
#mrg_final
#mrg_df.columns 

In [324]:
#Step 4
#before you reduced this to the top 15 items, how many entries 
#did you lose?
outer_mrg1 = pd.merge(ScimEn,energy,how='outer',left_on='Country',right_on='Country')
outer_mrg2 = pd.merge(outer_mrg1,GDP,how='outer',left_on='Country',right_on='Country Name')
len(outer_mrg2) - len(mrg_final)

298

In [325]:
#Step 5
#What is the average GDP over the last 10 years for each country?
gdp_cols = mrg_final[['2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']]
avgGDP = gdp_cols.mean(axis=1)
pd.Series(avgGDP)

Country
China                 6.348609e+12
Japan                 5.542208e+12
Russian Federation    1.565459e+12
Canada                1.660647e+12
Germany               3.493025e+12
India                 1.769297e+12
France                2.681725e+12
South Korea           1.106715e+12
Italy                 2.120175e+12
Spain                 1.418078e+12
Australia             1.164043e+12
Brazil                2.189794e+12
dtype: float64

In [326]:
#Step 6
#What is the mean Energy Supply per Capita?
supply=mrg_final['Energy Supply per Capita'].mean()
supply

152.91666666666666

In [327]:
#Step 7
#What country has the maximum % Renewable 
#and what is the percentage?
Renw = mrg_final['% Renewable']
MaxRenw = Renw.idxmax()
Renw[MaxRenw]
tup = (MaxRenw,Renw[MaxRenw])
tup

('Brazil', 69.64803)

In [328]:
#Step 8
#Create a new column that is the ratio of Self-Citations to Total Citations. 
#What is the maximum value for this new column, 
#and what country has the highest ratio?
ratio=mrg_final['Self-citations']/ mrg_final['Citations']
MaxRatio = ratio.idxmax()
ratio[MaxRatio]
tupl= (MaxRatio,ratio[MaxRatio])
tupl

('China', 0.6893126179389422)

In [329]:
#Step 9
#Create a new column with a 1 if the country's % Renewable 
#value is at or above the median for all countries, 
#and a 0 if the country's % Renewable value is below the median.
Renw = mrg_final['% Renewable']
RenwMid = Renw.median()
mrg_final['HighRenew'] = None
for i in range(len(Renw)):
    if Renw[i]>= RenwMid:
        mrg_final['HighRenew'] = 1
    else:
        mrg_final['HighRenew'] = 0

pd.Series(mrg_final['HighRenew'])

Country
China                 1
Japan                 1
Russian Federation    1
Canada                1
Germany               1
India                 1
France                1
South Korea           1
Italy                 1
Spain                 1
Australia             1
Brazil                1
Name: HighRenew, dtype: int64

In [330]:
#Step 10
#group the Countries by Continent, 
#then create a dateframe that displays the sample size 
#and the sum, mean, and std deviation for the estimated population.
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
#mrg_final


In [331]:
#len(frame) == number of countries in this continent
mrg_final['Population'] = mrg_final['Energy Supply'] / mrg_final['Energy Supply per Capita']
group_df = pd.DataFrame(columns = ['size','sum','mean','std'])
for group , frame in mrg_final.groupby(ContinentDict):
    group_df.loc[group] = [len(frame), frame['Population'].sum(),frame['Population'].mean(),frame['Population'].std()]
group_df

,size,sum,mean,std
Asia,4.0,2.821591e+09,7.053977e+08,7.138779e+08
Australia,1.0,2.331602e+07,2.331602e+07,NaN
Europe,5.0,3.940587e+08,7.881174e+07,3.813228e+07
North America,1.0,3.523986e+07,3.523986e+07,NaN
South America,1.0,2.059153e+08,2.059153e+08,NaN
